In [ ]:
pl.Config.set_tbl_rows(2000)

In [1]:
import polars as pl
from src.facilities_endpoint import fetch_facilities_data

df = await fetch_facilities_data()
if df is None:
    raise ValueError("df is None")
lf = df.lazy()
lf.collect()

primo_id,division_name,division_id,facility_name,route_name,foreman_name
i64,str,i64,str,str,str
480001,"""COLLEGE STATION""",19,"""AGGIELAND FAC""","""WEST TEXAS ROUTE""","""REYNOLDS, RYAN"""
69419,"""HOUSTON""",5,"""BAYOU CITY FAC""","""HUSTLE CITY ROUTE""","""COOPER, SHELDON"""
98743,"""AUSTIN""",12,"""ATX FAC""","""VIOLET ROUTE""","""BURTON, TIM"""
98750,"""AUSTIN""",12,"""CAPITAL CITY FAC""","""LEGEND CITY ROUTE""","""DEPP, JOHNNY"""
98761,"""DALLAS""",26,"""ANDERSON FAC""","""TRIPLE D ROUTE""","""SMITH, JOHN"""
…,…,…,…,…,…
98837,"""LUBBOCK""",40,"""FORT BEND FAC""","""HUB CITY ROUTE""","""STARK, TONY"""
98838,"""LUBBOCK""",40,"""FRANKLIN FAC""","""HUB CITY ROUTE""","""STARK, TONY"""
98839,"""LUBBOCK""",40,"""FREESTONE FAC""","""HUB CITY ROUTE""","""STARK, TONY"""


In [ ]:
lf = lf.with_columns(
    pl.when(pl.col("facility_name").is_null())
    .then(pl.lit("WEL"))
    .otherwise(pl.lit("FAC"))
    .alias("entity_type")
)
lf.collect()

In [ ]:
coalesced_lf = lf.with_columns(
    pl.col("entity_type"),
    pl.col("foreman_name"),
    pl.col("route_name"),
    pl.coalesce(pl.col("primo_prprty"), pl.col("fac_primo")).alias("primo_prprty"), 
    pl.coalesce(pl.col("division_id"), pl.col("divs")).alias("division_id"),
    pl.coalesce(pl.col("facility_name"), pl.col("completion_name")).alias("entity_name"),
    pl.coalesce(pl.col("routeid"), pl.col("route_id")).alias("route_id")
)

required_columns = ["primo_prprty", "division_id", "entity_name", "route_id", "entity_type", "foreman_name", "route_name"]
coalesced_lf = coalesced_lf.select(required_columns)
coalesced_lf.collect()